In [1]:
import pandas as pd

input_file = 'healthcare_noshows_appt.csv'

# Чтение CSV файла в DataFrame.
df = pd.read_csv(input_file)
df.head()

,PatientId,AppointmentID,Gender,ScheduledDay,AppointmentDay,Age,Neighbourhood,Scholarship,Hipertension,Diabetes,Alcoholism,Handcap,SMS_received,Showed_up,Date.diff
0,2.987250e+13,5642903,F,2016-04-29,2016-04-29,62,JARDIM DA PENHA,False,True,False,False,False,False,True,0
1,5.589978e+14,5642503,M,2016-04-29,2016-04-29,56,JARDIM DA PENHA,False,False,False,False,False,False,True,0
2,4.262962e+12,5642549,F,2016-04-29,2016-04-29,62,MATA DA PRAIA,False,False,False,False,False,False,True,0
3,8.679512e+11,5642828,F,2016-04-29,2016-04-29,8,PONTAL DE CAMBURI,False,False,False,False,False,False,True,0
4,8.841186e+12,5642494,F,2016-04-29,2016-04-29,56,JARDIM DA PENHA,False,True,True,False,False,False,True,0


In [2]:
# Отсортируем по дате приема
df = df.sort_values(by='AppointmentDay')

In [3]:
# Определим долю неявок для каждого пациента за время, предшествующее текущему приему
df['no_show'] = ~df['Showed_up']
df['no_show_cumsum'] = df.groupby('PatientId')['no_show'].cumsum() - df['no_show']
df['Appointment_cumcount'] = df.groupby('PatientId').cumcount()
df['no_show_ratio'] = df['no_show_cumsum'] / df['Appointment_cumcount'] * 100

In [4]:
df['no_show_ratio'] = df['no_show_ratio'].fillna(0)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 106987 entries, 0 to 99545
Data columns (total 19 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   PatientId             106987 non-null  float64
 1   AppointmentID         106987 non-null  int64  
 2   Gender                106987 non-null  object 
 3   ScheduledDay          106987 non-null  object 
 4   AppointmentDay        106987 non-null  object 
 5   Age                   106987 non-null  int64  
 6   Neighbourhood         106987 non-null  object 
 7   Scholarship           106987 non-null  bool   
 8   Hipertension          106987 non-null  bool   
 9   Diabetes              106987 non-null  bool   
 10  Alcoholism            106987 non-null  bool   
 11  Handcap               106987 non-null  bool   
 12  SMS_received          106987 non-null  bool   
 13  Showed_up             106987 non-null  bool   
 14  Date.diff             106987 non-null  int64  
 15  no_sho

### Посещения

In [8]:
df_appointment = df[['PatientId', 'AppointmentID', 
                     'ScheduledDay', 'AppointmentDay']]

In [9]:
df_appointment.columns = ['patient_id', 'appointment_id', 'scheduled_date', 'appointment_date']

In [10]:
df_appointment = df_appointment.iloc[:50, :]

In [11]:
doctor_names = ["Иванов А.В.", "Смирнов О.П.", "Петров И.М.", "Соколов Д.Н.", "Васильев Е.С."] * 10
slots = [num for num in range(1, 11) for _ in range(5)]

In [12]:
df_appointment['doctor_name'] = doctor_names
df_appointment['slot_id'] = slots

In [13]:
df_appointment.info()

<class 'pandas.core.frame.DataFrame'>
Index: 50 entries, 0 to 2112
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   patient_id        50 non-null     float64
 1   appointment_id    50 non-null     int64  
 2   scheduled_date    50 non-null     object 
 3   appointment_date  50 non-null     object 
 4   doctor_name       50 non-null     object 
 5   slot_id           50 non-null     int64  
dtypes: float64(1), int64(2), object(3)
memory usage: 2.7+ KB


In [14]:
df_appointment.to_csv("Appointments.csv", index=False)

### Пациенты

In [15]:
df_patient = df.drop_duplicates(subset='PatientId', keep='last')

In [16]:
df_patient = df_patient.drop(columns=['AppointmentID', 'Showed_up', 'no_show', 
                                      'Date.diff', 'AppointmentDay', 'ScheduledDay'])

In [18]:
df_patient.columns = ['patient_id', 'gender', 'age', 'neighbourhood',
                      'scholarship', 'hipertension', 'diabetes', 
                      'alcoholism', 'handcap', 'sms_received', 
                      'no_show_cumsum', 'appointment_cumcount', 'no_show_ratio']

In [19]:
df_patient.info()

<class 'pandas.core.frame.DataFrame'>
Index: 60270 entries, 2066 to 99545
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   patient_id            60270 non-null  float64
 1   gender                60270 non-null  object 
 2   age                   60270 non-null  int64  
 3   neighbourhood         60270 non-null  object 
 4   scholarship           60270 non-null  bool   
 5   hipertension          60270 non-null  bool   
 6   diabetes              60270 non-null  bool   
 7   alcoholism            60270 non-null  bool   
 8   handcap               60270 non-null  bool   
 9   sms_received          60270 non-null  bool   
 10  no_show_cumsum        60270 non-null  int64  
 11  appointment_cumcount  60270 non-null  int64  
 12  no_show_ratio         60270 non-null  float64
dtypes: bool(6), float64(2), int64(3), object(2)
memory usage: 4.0+ MB


In [20]:
df_patient.to_csv("Patients.csv", index=False)